In [29]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "GonSNHU1234"

# connection variables
USER = "aacuser"
PASS = "GonSNHU1234"
HOST = 'nv-desktop-services.apporto.com'
PORT = 34053
DB = 'AAC'
COL = 'animals'

shelter = AnimalShelter(username, password, HOST, PORT, DB, COL)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))
print(df.columns)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GSLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# Define the layout of the app
app.layout = html.Div([
    # Centered title
    html.Center(html.B(html.H1('Gon*CS-340 Dashboard'))),
    html.Hr(),

    # Image display (Grazioso Salvare's logo) - remember to replace with actual image path
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),

    # Filtering options (Example: Radio buttons for filtering by animal type)
     # Updated filtering options
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'ALL'},
                {'label': 'Dogs', 'value': 'Dog'},
                {'label': 'Cats', 'value': 'Cat'},
                {'label': 'Birds', 'value': 'Bird'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster Tracking'}
            ],
            value='ALL',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),

    # Layout for charts and maps
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback to update the data table based on the selected filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'ALL':
        query = {}
    elif filter_type in ['Water Rescue', 'Mountain Wilderness Rescue', 'Disaster Tracking']:
        query = {"rescue_type": filter_type}
    else:
        query = {"animal_type": filter_type}

    df = pd.DataFrame.from_records(shelter.read(query))


    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    
    return df.to_dict('records')

# Callback to update the graph based on the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if len(dff) == 0:
        return [dcc.Graph(figure={})]

    fig = px.pie(dff, names='breed', title='Preferred Animals by Breed')
    return [dcc.Graph(figure=fig)]

# Callback to highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Callback to update the geolocation map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(viewData) == 0:
        # Return an empty map or a message if there is no data
        return [html.Div("No data available for the selected filter.")]

    dff = pd.DataFrame.from_dict(viewData)
    
    if len(index) == 0:
        return [html.Div("No row selected.")]

    row = index[0]

    # Ensure that the row index is within the bounds of the DataFrame
    if row >= len(dff):
        return [html.Div("Selected row index is out of bounds.")]

    # Assuming the latitude and longitude are in specific columns
    lat_col = 13  # Adjust this index based on your DataFrame structure
    lon_col = 14  # Adjust this index based on your DataFrame structure

    # Check if the latitude and longitude columns exist
    if lat_col >= len(dff.columns) or lon_col >= len(dff.columns):
        return [html.Div("Latitude or Longitude column not found.")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, lat_col], dff.iloc[row, lon_col]], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Adjust this index based on your DataFrame structure
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])  # Adjust this index based on your DataFrame structure
                ])
            ])
        ])
    ]

app.run_server(debug=True)

Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', 'age'],
      dtype='object')
Dash app running on http://127.0.0.1:16419/
